<a href="https://colab.research.google.com/github/eulerianKnight/semantic_search/blob/main/Haystack_Semantic_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab,faiss-gpu]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 8.7 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-kyhfoz4k/farm-haystack_b4f925ba653a4ac8b08de89082062f9b
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-kyhfoz4k/farm-haystack_b4f925ba653a4ac8b08de89082062f9b
  Resolved https://github.com/deepset-ai/haystack.git to commit 96bb9b5905dcef61264f40ead6eea1ecaf2991b7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 53.

In [2]:
import pandas as pd
from haystack.document_stores import FAISSDocumentStore
from haystack.nodes import EmbeddingRetriever

## Data

In [3]:
!git clone https://github.com/yashprakash13/datasets.git

Cloning into 'datasets'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 3 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.


In [4]:
arxiv_data = pd.read_csv('/content/datasets/arxiv_short.csv')
arxiv_data.head()

,authors,title,abstract
0,"C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-P. Yuan",Calculation of prompt diphoton production cross sections at Tevatron and\n ...,A fully differential calculation in perturbative quantum chromodynamics is...
1,Ileana Streinu and Louis Theran,Sparsity-certifying Graph Decompositions,"We describe a new algorithm, the $(k,\ell)$-pebble game with colors, and u..."
2,Hongjun Pan,The evolution of the Earth-Moon system based on the dark matter field\n flu...,The evolution of Earth-Moon system is described by the dark matter field\n...
3,David Callan,A determinant of Stirling cycle numbers counts unlabeled acyclic\n single-s...,We show that a determinant of Stirling cycle numbers counts unlabeled acyc...
4,Wael Abu-Shammala and Alberto Torchinsky,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\alpha}$,"In this paper we show how to compute the $\Lambda_{\alpha}$ norm, $\alpha\..."


## Initialise Document store and retriever

In [5]:
document_store_faiss = FAISSDocumentStore(faiss_index_factory_str="Flat", 
                                          return_embedding=True)

In [6]:
retriever_faiss = EmbeddingRetriever(document_store_faiss, 
                                     embedding_model='distilroberta-base-msmarco-v2', 
                                     model_format='sentence_transformers')

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/683 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

## Build Document Store

In [8]:
document_store_faiss.delete_all_documents()

document_store_faiss.write_documents(
arxiv_data[['authors', 'title', 'abstract']].rename(columns={ 
'title':'name',
'author' : 'author',
'abstract':'content'
}
).to_dict(orient='records'))

                1. delete_all_documents() method is deprecated, please use delete_documents method
                For more details, please refer to the issue: https://github.com/deepset-ai/haystack/issues/1045
                


Writing Documents:   0%|          | 0/50000 [00:00<?, ?it/s]

### Pass Document Store to Retriever

In [9]:
document_store_faiss.update_embeddings(retriever=retriever_faiss)

Updating Embedding:   0%|          | 0/49924 [00:00<?, ? docs/s]

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

Batches:   0%|          | 0/311 [00:00<?, ?it/s]

### Fetch Relevant Documents

In [14]:
def get_results(query, retriever, n_res = 10):
    return [(item.content, item.to_dict()['meta']) for item in retriever.retrieve(query, top_k = n_res)]

In [16]:
query = 'Turbulent flow in contrast to laminar flow'
print('Results: ')
res = get_results(query, retriever_faiss)
for r in res:
    print(r)

Results: 


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

('  The work reported below is a first of its kind study of the properties of\nturbulent flow without strong mean shear in a Newtonian fluid in proximity of\nthe turbulent/non-turbulent interface, with emphasis on the small scale\naspects. The main tools used are a three-dimensional particle tracking system\n(3D-PTV) allowing to measure and follow in a Lagrangian manner the field of\nvelocity derivatives and direct numerical simulations (DNS). The comparison of\nflow properties in the turbulent (A), intermediate (B) and non-turbulent (C)\nregions in the proximity of the interface allows for direct observation of the\nkey physical processes underlying the entrainment phenomenon. The differences\nbetween small scale strain and enstrophy are striking and point to the definite\nscenario of turbulent entrainment via the viscous forces originating in strain.\n', {'authors': 'M. Holzner, A. Liberzon, N. Nikitin, W. Kinzelbach, A. Tsinober', 'name': 'Small scale aspects of flows in proximity o